In [1]:
import pandas as pd
import pathlib
import sys
from sklearn.preprocessing import MinMaxScaler
import joblib
import torch
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt

sys.path.insert(0, "../utils/")
from data_loader import load_model_data
from pinwheels import compute_and_plot_latent_scores, assign_unique_latent_dims
from model_utils import extract_latent_dims

sys.path.append("../5.drug-dependency")
from utils import load_utils

In [2]:
data_directory = pathlib.Path("../0.data-download/data").resolve(strict=True)
dependency_file = pathlib.Path(f"{data_directory}/CRISPRGeneEffect.parquet").resolve(strict=True)
gene_dict_file = pathlib.Path(f"{data_directory}/CRISPR_gene_dictionary.parquet").resolve(strict=True)

In [3]:
# Load metadata
metadata_df_dir = pathlib.Path("../0.data-download/data/metadata_df.parquet").resolve(strict=True)
metadata = pd.read_parquet(metadata_df_dir)
print(metadata.shape)

#Load dependency data
dependency_df, gene_dict_df = load_model_data(dependency_file, gene_dict_file)
dependency_df.head()

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply the scaler to the numeric columns
dependency_df[dependency_df.select_dtypes(include=['float64', 'int']).columns] = scaler.fit_transform(
    dependency_df.select_dtypes(include=['float64', 'int'])
)

(958, 3)
(1150, 18444)


In [4]:
train_and_test_subbed_dir = pathlib.Path("../0.data-download/data/train_and_test_subbed.parquet").resolve(strict=True)
train_and_test_subbed = pd.read_parquet(train_and_test_subbed_dir)

# Convert DataFrame to NumPy and then Tensor
train_test_array = train_and_test_subbed.to_numpy()
train_test_tensor = torch.tensor(train_test_array, dtype=torch.float32)

#Create TensorDataset and DataLoader
tensor_dataset = TensorDataset(train_test_tensor)
train_and_test_subbed_loader = DataLoader(tensor_dataset, batch_size=32, shuffle=False)

In [5]:
corum_dir = pathlib.Path("../4.gene-expression-signatures/gsea_results/combined_z_matrix_gsea_results_corum.parquet")
corum_df = pd.read_parquet(corum_dir)

reactome_dir = pathlib.Path("../4.gene-expression-signatures/gsea_results/combined_z_matrix_gsea_results.parquet")
reactome_df = pd.read_parquet(reactome_dir)

In [ ]:
# Define cut-offs
lfc_cutoff = 0.584
fdr_cutoff = 0.05

# Filter data for significant results
significant_corum_df = corum_df[
    (corum_df['gsea_es_score'].abs() > lfc_cutoff) & 
    (corum_df['p_value'] < fdr_cutoff)
]

significant_reactome_df = reactome_df[
    (reactome_df['gsea_es_score'].abs() > lfc_cutoff) & 
    (reactome_df['p_value'] < fdr_cutoff)
]

In [6]:
# Define the location of the saved models and output directory for results
model_save_dir = pathlib.Path("../4.gene-expression-signatures/saved_models")
output_dir = pathlib.Path("results")
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Placeholder for storing all latent representations
latent_dfs = []
cache_file = pathlib.Path("./results/combined_latent_df.parquet")

# Check if the cached file exists
if cache_file.exists():
    print(f"Loading cached latent representations from {cache_file}")
    combined_latent_df = pd.read_parquet(cache_file)

else:
    # Iterate over all saved model files
    for model_file in model_save_dir.glob("*.joblib"):
        model_file_name = model_file.stem
        try:
            parts = model_file_name.split("_")
            model_name = parts[0]
            num_components = int(parts[3])  # Example: Extract number of components
            init = int(parts[7])  # Extract initialization value
            seed = int(parts[9])  # Extract seed value
        except (IndexError, ValueError):
            print(f"Skipping file {model_file} due to unexpected filename format.")
            continue

        # Load the model
        print(f"Loading model from {model_file}")
        try:
            model = joblib.load(model_file)
        except Exception as e:
            print(f"Failed to load model from {model_file}: {e}")
            continue
        

        # Extract latent dimensions
        latent_df = extract_latent_dims(model_name, model, dependency_df, train_and_test_subbed_loader, metadata)

        # Add metadata to the latent dataframe
        latent_df["model"] = model_name
        latent_df["latent_dim_total"] = num_components
        latent_df["init"] = init
        latent_df["seed"] = seed

        # Move metadata columns to the front
        metadata_columns = ["model", "latent_dim_total", "init", "seed"]
        latent_df = latent_df.loc[:, ~latent_df.columns.duplicated()]
        latent_df.columns = latent_df.columns.astype(str)
        latent_df = latent_df[metadata_columns + [col for col in latent_df.columns if col not in metadata_columns]]
        latent_dfs.append(latent_df)

    # Combine all latent representations into one dataframe
    combined_latent_df = pd.concat(latent_dfs, ignore_index=True)
    combined_latent_df.to_parquet(cache_file)

Loading model from ../4.gene-expression-signatures/saved_models/betatcvae_latent_dims_5_trial_20_init_2_seed_1999415996.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_14_trial_42_init_4_seed_1154917792.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betatcvae_latent_dims_20_trial_10_init_1_seed_844030991.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_3_trial_41_init_4_seed_87832445.joblib
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_150_trial_30_init_1_seed_1559165653.joblib
Loading model from ../4.gene-expression-signatures/saved_models/pca_latent_dims_35_trial_non-optuna_init_0_seed_0.joblib


/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(
/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator FastICA from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
http

Loading model from ../4.gene-expression-signatures/saved_models/ica_latent_dims_50_trial_non-optuna_init_0_seed_0.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_16_trial_43_init_3_seed_2176741505.joblib


/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but FastICA was fitted without feature names
  warnings.warn(
/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Loading model from ../4.gene-expression-signatures/saved_models/betatcvae_latent_dims_150_trial_43_init_1_seed_1559165653.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_100_trial_44_init_4_seed_4073875038.joblib
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_12_trial_49_init_4_seed_1154917792.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betatcvae_latent_dims_35_trial_12_init_0_seed_501796030.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betatcvae_latent_dims_2_trial_35_init_3_seed_4066919116.joblib
Loading model from ../4.gene-expression-signatures/saved_models/pca_latent_dims_18_trial_non-optuna_init_0_seed_0.joblib


...

Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_18_trial_49_init_0_seed_501796030.joblib
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_25_trial_16_init_2_seed_1507182516.joblib
Loading model from ../4.gene-expression-signatures/saved_models/pca_latent_dims_7_trial_non-optuna_init_0_seed_0.joblib


/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(
/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator NMF from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Loading model from ../4.gene-expression-signatures/saved_models/betatcvae_latent_dims_4_trial_21_init_0_seed_950231792.joblib
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_200_trial_32_init_4_seed_4073875038.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_18_trial_12_init_0_seed_501796030.joblib
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_4_trial_49_init_1_seed_3685236136.joblib
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_14_trial_20_init_0_seed_501796030.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_150_trial_19_init_3_seed_3783068897.joblib
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_4_trial_22_init_3_seed_4066919116.joblib
Loading model from ../4.gene-expression-signatures/saved_models/nmf_latent_dims_2_trial_non-optuna_init_0_seed_

/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but NMF was fitted without feature names
  warnings.warn(


Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_3_trial_11_init_0_seed_950231792.joblib


In [9]:
drug_dir = pathlib.Path("../5.drug-dependency/results/combined_latent_drug_correlations.parquet")
drug_df = pd.read_parquet(drug_dir)

# Load PRISM data
top_dir = "../5.drug-dependency"
data_dir = "data"

prism_df, prism_cell_df, prism_trt_df = load_utils.load_prism(
    top_dir=top_dir,
    data_dir=data_dir,
    secondary_screen=False,
    load_cell_info=True,
    load_treatment_info=True,
)

In [10]:
# Merge drug_df with prism_trt_df to replace drug IDs with drug names
drug_df = drug_df.merge(prism_trt_df[['column_name', 'name']], 
                        left_on='drug', 
                        right_on='column_name', 
                        how='left')

# Drop the redundant column_name column
drug_df = drug_df.drop(columns=['column_name'])

# Display the updated dataframe
drug_df.head()

,z,full_model_z,model,init,drug,pearson_correlation,p_value,shuffled,name
0,0,5,betatcvae,2,BRD-A00077618-236-07-6::2.5::HTS,0.014884,0.764663,False,8-bromo-cGMP
1,0,5,betatcvae,2,BRD-A00100033-001-08-9::2.5::HTS,0.007519,0.879653,False,nifurtimox
2,0,5,betatcvae,2,BRD-A00147595-001-01-5::2.5::HTS,0.037139,0.454939,False,balaglitazone
3,0,5,betatcvae,2,BRD-A00218260-001-03-4::2.5::HTS,0.060939,0.221635,False,flutrimazole
4,0,5,betatcvae,2,BRD-A00376169-001-01-6::2.5::HTS,0.050305,0.313157,False,KBG


In [ ]:
# Define cut-offs
corr_cutoff = 0.15
pval_cutoff = 0.05

# Filter data for significant results
significant_drug_df = drug_df[
    (drug_df['pearson_correlation'].abs() > corr_cutoff) & 
    (drug_df['p_value'] < pval_cutoff)
]

In [ ]:
reactome_max = assign_unique_latent_dims(significant_reactome_df, score_col="gsea_es_score", target_col="reactome_pathway")
corum_max = assign_unique_latent_dims(significant_corum_df, score_col="gsea_es_score", target_col="reactome_pathway")
drug_max = assign_unique_latent_dims(significant_drug_df, score_col="pearson_correlation", target_col="name")

In [11]:
# Loop through each unique ModelID to process and plot
for model_id in combined_latent_df['ModelID'].unique():
    compute_and_plot_latent_scores(model_id, combined_latent_df, reactome_max, "reactome_pathway", "gsea_es_score", "Multi-Gene Dependency")

ACH-000499
ACH-001668
ACH-001211
ACH-000997
ACH-001745
ACH-001650
ACH-000991
ACH-001184
ACH-000976
ACH-001402
ACH-002680
ACH-000048
ACH-001715
ACH-002647
ACH-001283
ACH-000588
ACH-000026
ACH-000878
ACH-000962
ACH-002280
ACH-000366
ACH-000546
ACH-000845
ACH-000741
ACH-001819
ACH-000405
ACH-001353
ACH-000190
ACH-000903
ACH-000454
ACH-001422
ACH-000406
ACH-000040
ACH-001496
ACH-001450
ACH-000817
ACH-000288
ACH-000975
ACH-000652
ACH-001990
ACH-001536
ACH-001515
ACH-002030
ACH-000824
ACH-002033
ACH-002471
ACH-000106
ACH-001038
ACH-000571
ACH-000971
ACH-000607
ACH-000233
ACH-001098
ACH-000082
ACH-000928
ACH-000663
ACH-001442
ACH-002016
ACH-002669
ACH-000756
ACH-002046
ACH-000694
ACH-000577
ACH-000489
ACH-000163
ACH-000183
ACH-002526
ACH-000811
ACH-001653
ACH-000872
ACH-000800
ACH-002029
ACH-001605
ACH-000025
ACH-000421
ACH-001031
ACH-001028
ACH-001454
ACH-002847
ACH-002143
ACH-000516
ACH-000139
ACH-001547
ACH-000202
ACH-000416
ACH-000738
ACH-000456
ACH-000596
ACH-000290
ACH-000167
ACH-000906

In [ ]:
# Loop through each unique ModelID to process and plot
for model_id in combined_latent_df['ModelID'].unique():
    compute_and_plot_latent_scores(model_id, combined_latent_df, corum_max, "reactome_pathway", "gsea_es_score", "Multi-Gene Dependency")

In [12]:
# Loop through each unique ModelID to process and plot
for model_id in combined_latent_df['ModelID'].unique():
    compute_and_plot_latent_scores(model_id, combined_latent_df, drug_max, "name", "pearson_correlation", "Drug")

ACH-000499
ACH-001668
ACH-001211
ACH-000997
ACH-001745
ACH-001650
ACH-000991
ACH-001184
ACH-000976
ACH-001402
ACH-002680
ACH-000048
ACH-001715
ACH-002647
ACH-001283
ACH-000588
ACH-000026
ACH-000878
ACH-000962
ACH-002280
ACH-000366
ACH-000546
ACH-000845
ACH-000741
ACH-001819
ACH-000405
ACH-001353
ACH-000190
ACH-000903
ACH-000454
ACH-001422
ACH-000406
ACH-000040
ACH-001496
ACH-001450
ACH-000817
ACH-000288
ACH-000975
ACH-000652
ACH-001990
ACH-001536
ACH-001515
ACH-002030
ACH-000824
ACH-002033
ACH-002471
ACH-000106
ACH-001038
ACH-000571
ACH-000971
ACH-000607
ACH-000233
ACH-001098
ACH-000082
ACH-000928
ACH-000663
ACH-001442
ACH-002016
ACH-002669
ACH-000756
ACH-002046
ACH-000694
ACH-000577
ACH-000489
ACH-000163
ACH-000183
ACH-002526
ACH-000811
ACH-001653
ACH-000872
ACH-000800
ACH-002029
ACH-001605
ACH-000025
ACH-000421
ACH-001031
ACH-001028
ACH-001454
ACH-002847
ACH-002143
ACH-000516
ACH-000139
ACH-001547
ACH-000202
ACH-000416
ACH-000738
ACH-000456
ACH-000596
ACH-000290
ACH-000167
ACH-000906